In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

import keras
import tensorflow as tf
from keras import backend as K 
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from tensorflow.python.keras.engine.base_layer import Layer
from keras import optimizers
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D ,Flatten, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

import math
import numpy as np
import pickle
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_train = x_train.astype('float32') / 255.
X_test = x_test.astype('float32') / 255.
X_train = np.reshape(X_train, (len(X_train), 28, 28, 1))    # adapt this if using 'channels_first' image data format
X_test = np.reshape(X_test, (len(X_test), 28, 28, 1))       # adapt this if using 'channels_first' image data format

X_train = np.reshape(X_train, (len(X_train), 28,28, 1))    # adapt this if using 'channels_first' image data format
X_test = np.reshape(X_test, (len(X_test), 28,28, 1))       # adapt this if using 'channels_first' image data format



In [3]:
b = np.zeros((len(y_test),10))
for i in range(0,len(y_test)):
    b[i][y_test[i]]+=1
y_test = b

a = np.zeros((len(y_train),10))
for i in range(0,len(y_train)):
    a[i][y_train[i]]+=1
y_train = a  



In [4]:
input_img = Input(shape=(28, 28, 1))    # adapt this if using 'channels_first' image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x1 = MaxPooling2D((2, 2), padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x1)
flat = Flatten()(encoded)
decoded = Dense(10, activation='softmax')(flat)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='Adam', loss='binary_crossentropy')

autoencoder.fit(X_train, y_train, epochs=10, batch_size=128, shuffle=True, validation_data=(X_test, y_test),
                callbacks=[TensorBoard(log_dir='conv_autoencoder')], verbose=1)


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 23s 376us/step - loss: 0.0940 - val_loss: 0.0391
Epoch 2/10
60000/60000 [==============================] - 32s 534us/step - loss: 0.0332 - val_loss: 0.0265
Epoch 3/10
60000/60000 [==============================] - 24s 404us/step - loss: 0.0242 - val_loss: 0.0214
Epoch 4/10
60000/60000 [==============================] - 20s 329us/step - loss: 0.0200 - val_loss: 0.0177
Epoch 5/10
60000/60000 [==============================] - 19s 312us/step - loss: 0.0175 - val_loss: 0.0154
Epoch 6/10
60000/60000 [==============================] - 24s 395us/step - loss: 0.0158 - val_loss: 0.0151
Epoch 7/10
60000/60000 [==============================] - 24s 405us/step - loss: 0.0146 - val_loss: 0.0143
Epoch 8/10
60000/60000 [==============================] - 20s 333us/step - loss: 0.0138 - val_loss: 0.0127
Epoch 9/10
60000/60000 [==============================] - 16s 272us/step - loss: 0.0129 - val_

In [5]:
layer_weights = []
for layer in autoencoder.layers: 
    if(len(layer.get_weights())>0):
        layer_weights.append((layer.get_weights())[0])


wei1 = layer_weights[0]
wei2 = layer_weights[1]

    

In [6]:
def ReLU(X):
    return tf.maximum(X, np.zeros_like(X))

def softmax(X):
    return tf.exp(X)/tf.reduce_sum(tf.exp(X), 1)

def tanh(X):
    return tf.tanh(X)

def sigmoid(X):
    return 1 / (1 + tf.exp(-X))

In [7]:
def myConv2D(X, W, filters, kernel_size, strides=1, padding='valid', activation=None, bias=0):

    if isinstance(kernel_size, int):
        ker_h, ker_w = kernel_size, kernel_size
    elif isinstance(kernel_size, tuple):
        ker_h, ker_w = kernel_size
    else:
        raise ValueError('Kernel size should be tuple of ints or an int.')
        
        
    batch , inp_h, inp_w, inp_c = X.shape
    if padding == 'valid':
        pad = 0
    elif padding == 'same':
        pad = (ker_h - 1)//2
    else:
        raise ValueError('Padding '+padding+' is not defined')
    
    conv_h = (inp_h + 2*pad - ker_h)//strides + 1
    conv_w = (inp_w + 2*pad - ker_w)//strides + 1
    
    X_pad = tf.pad(X, [[0,0],[pad,pad],[pad,pad],[0,0]])
    
    convolve = []
    for x in range(conv_h):
        for y in range(conv_w):
            c = tf.slice(X_pad, [0, x*strides, y*strides, 0], [-1, ker_h, ker_w, -1])
            convolve.append(c)
    
    X_mat = tf.reshape(tf.stack(convolve), [-1, ker_w*ker_h])
#     print("X_mat: ", X_mat.shape)
    W = W[:,:,0,:]
    W_mat = tf.reshape(W, [ker_h*ker_w, filters])
    
    b = tf.zeros([filters])+bias
    
    conv = tf.matmul(X_mat, W_mat) + b
    conv = tf.transpose(tf.reshape(conv,[conv_h,conv_w,-1,filters]), [2,0,1,3])
    
    if activation:
        activation = activation.lower()
        if activation == 'relu':
            conv = ReLU(conv)
        elif activation == 'softmax':
            conv = softmax(conv)
        elif activation == 'tanh':
            conv = tanh(conv)
        elif activation == 'sigmoid':
            conv = sigmoid(conv)    
        else:
            raise ValueError('Unknown activation')
            
    
    return conv
    
# x1 = myConv2D(input_batch, 16, (3,3), padding = 'same', activation = 'relu')
# print(x1.shape)

In [8]:
def myMaxPool2D(X, kernel_size, strides=2, padding='valid'):
    if isinstance(kernel_size, int):
        ker_h, ker_w = kernel_size, kernel_size
    elif isinstance(kernel_size, tuple):
        ker_h, ker_w = kernel_size
    else:
        raise ValueError('Kernel size should be tuple of ints or an int.')
        
    _, inp_h, inp_w, inp_c = X.shape
    if padding == 'valid':
        pad = 0
    elif padding == 'same':
        pad = (ker_h - 1)//2
    else:
        raise ValueError('Padding '+padding+' is not defined')
    
    pool_h = (inp_h + 2*pad - ker_h)//strides + 1 
    pool_w = (inp_w + 2*pad - ker_w)//strides + 1
    
    X_pad = tf.pad(X, [[0,0],[pad,pad],[pad,pad],[0,0]])
    
    pooling = []
    for x in range(pool_h):
        for y in range(pool_w):
            p = tf.slice(X_pad, [0, x*strides, y*strides, 0], [-1, ker_h, ker_w, -1])
            pooling.append(p)
    
    X_mat = tf.reshape(tf.stack(pooling), [-1, ker_w*ker_h])
    
    pool = tf.reduce_max(tf.reshape(X_mat, [pool_h, pool_w, -1, ker_h*ker_w, int(inp_c)]), axis = 3)
    return tf.transpose(pool, [2,0,1,3])

# x2 = myMaxPool2D(x1, (2,2), strides = 2, padding='valid')
# print(x2.shape)

In [9]:
def myUpSampling2D(X, kernel_size, data_format='channels_last', interpolation='nearest'):
    if isinstance(kernel_size, int):
        ker_h, ker_w = kernel_size, kernel_size
    elif isinstance(kernel_size, tuple):
        ker_h, ker_w = kernel_size
    else:
        raise ValueError('Kernel size should be tuple of ints or an int.')
        
    if (data_format=='channels_last'):
        n, inp_h, inp_w, inp_c = X.shape
    elif (data_format=='channels_first'):
        n, inp_c, inp_h, inp_w = X.shape
    else:
        raise ValueError('Data format is not valid.')
    
    print(type(n))
    out_h = inp_h*ker_h
    out_w = inp_w*ker_w
    
    l = []
    img = 0
    
    while img < int(n):
        i = 0
        while i < int(inp_h):
            temp = []
            j = 0
            while j < int(inp_w):
                p = tf.slice(X, [img,i,j,0], [1,1,1,1])
                for x in range(ker_w):
                    temp.append(p)
                j+=1
            for x in range(ker_h):
                for c in temp:
                    l.append(c)
            i+=1
        img+=1
    X_mat = tf.reshape(tf.stack(l), [1, out_h, out_w])
    
    l = []
    for i in range(int(inp_c)):
        l.append(X_mat)
        
    X_mat = tf.reshape(tf.stack(l), [1, out_h, out_w, int(inp_c)])
    
#     print(X_mat.shape)
    return X_mat
            
# myUpSampling2D(x2, (2,2))

In [10]:
def fully_connected(pool_out):
    flattened_vector = pool_out.flatten()
    return np.matmul(flattened_vector, wei2)

In [11]:
input_img = X_test
x = myConv2D(input_img, wei1, 16, (3,3), activation = 'relu', padding = 'same')
print("hai1",x.shape)
x = myMaxPool2D(x, (2,2), padding = 'same')
print("hai2",x.shape)
encoded1 = myMaxPool2D(x, (2,2), padding = 'same')



hai1 (10000, 28, 28, 16)
hai2 (10000, 14, 14, 16)


In [12]:
import tensorflow as tf
W1 = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
array = encoded1.eval(session=sess)
# print (array)

In [13]:
pred=[]
for i in array:
  pred.append(fully_connected(i))

y_pred1 = np.array(pred)
y_pred1.shape 

(10000, 10)

In [15]:
y_pred = autoencoder.predict(X_test)


In [16]:
print(roc_auc_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred1))

0.99969230622
0.997107608775
